# Evaluation of dataset features

In [ ]:
layer_name = 'CallTypeLayer' 

In [ ]:
import pandas as pd
from pandas import DataFrame

df: DataFrame = pd.read_csv(f'output/cluster_metrics/data/{layer_name}.csv', index_col=0)

## Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(df)[:,:-1]

In [ ]:
pd.DataFrame(data=X, columns=df.columns[:-1]).head(2)

In [ ]:
df.head(2)

## Principal Components

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
# compute scores in Xp
Xp = pca.fit_transform(X)
# get loadings and the ammount of variance explained by each component 
c, v = pca.components_, pca.explained_variance_

loadings = pd.DataFrame(data=c, columns=df.columns[:-1])

# print("Loadings of the first component: ({:.3f}, {:.3f})".format(c[0,0], c[0,1]))
# print("Loadings of the second component:  ({:.3f}, {:.3f})".format(c[1,0], c[1,1]))

In [ ]:
loadings

In [ ]:
loadings.apply(lambda col: col != 0).sum()

In [ ]:
import matplotlib.pyplot as plt

l = [(-8.657802, 41.160978), (-8.65782, 41.160969), (-8.657838, 41.16096)]

plt.plot([e[0] for e in l], [e[1] for e in l])